In [1]:
import numpy as np
import pandas as pd
import os,sys

# reading positive and negative csv files into dataframe
pos_file = "pos_feat_vec.csv"
neg_file = "neg_feat_vec.csv"

pos = pd.read_csv(pos_file, header = 0)
neg = pd.read_csv(neg_file, header = 0)
Data = pd.concat([pos, neg])

# shuffling the data frames
from sklearn.utils import shuffle

data = shuffle(Data, random_state=0)
names = data.ix[:,0]
labels = data.ix[:, len(data.columns) - 1]
features = data.ix[:, 1: len(data.columns) - 1]

features = features.as_matrix()
labels = labels.as_matrix()

In [2]:
#reading test dataset
test_file = "test_feat_vec.csv"
test_product = "test_product_list.txt"
test = pd.read_csv(test_file, header = 0)
names_position =  test.ix[:,0:3]
test_feature = test.ix[:, 3: len(test.columns)]

true_product = []
start = []
end = []
with open(test_product) as f:
    for line in f:
        words =  line.strip('\n').split(' ')
        start.append(words[len(words)-2])
        end.append(words[len(words)-1])
        true_product.append(' '.join(words[0:len(words)-2]))

In [3]:
true_product[0:5], start[0:5], end[0:5], names_position.dtypes

(['Acer Aspire 5755-6482',
  'Lenovo IdeaPad V570-1066AJU',
  'Asus U56E-BBL6',
  'Toshiba Satellite P755-S5215',
  'Samsung NP300E5A-A01UB'],
 ['119', '4901', '4974', '5051', '5122'],
 ['140', '4928', '4988', '5079', '5144'],
 product_name    object
 start            int64
 end              int64
 dtype: object)

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
rf = RandomForestClassifier(n_estimators=10)
fold = 10

accuracy = cross_val_score(rf, features, labels, cv=fold)
precision = cross_val_score(rf, features, labels, cv=fold, scoring='precision')
recall = cross_val_score(rf, features, labels, cv=fold, scoring='recall')

print('Accuracy: ', accuracy.sum() / fold)
print('Precision: ', precision.sum() / fold)
print('Recall: ', recall.sum() / fold)

('Accuracy: ', 0.91542424981042136)
('Precision: ', 0.90752400639916808)
('Recall: ', 0.91431053093166048)


In [5]:
rf.fit(features,labels)
prediction = rf.predict(test_feature)
prediction

array([0, 1, 0, ..., 0, 1, 0], dtype=int64)

In [120]:
with open('prediction_new.txt','w') as f:
    for i, row in names_position.iterrows():
            f.write('\t'.join([row[0],str(prediction[i])]))
            f.write('\n')
f.close()

In [6]:
Tp = 0
Tn = 0
Fp = 0
Fn = 0

for i, row in names_position.iterrows():
    if row[0] in true_product:
        index = true_product.index(row[0])
        if (start[index] == str(row[1]) and end[index] == str(row[2])):
            if (prediction[i] == 1):
                Tp=Tp+1
            else:
                Fn=Fn+1
        else:
            if (prediction[i] == 1):
                Fp=Fp+1
            else:
                Tn=Tn+1
    else:
        if (prediction[i] == 1):
            Fp=Fp+1
        else:
            Tn=Tn+1
Tp,Tn,Fp,Fn, (Tp+Tn+Fp+Fn)   



(337, 5778, 1123, 36, 7274)

In [7]:
precision = (float(Tp)/float(Tp+Fp))
recall = (float(Tp)/float(Tp+Fn))
precision, recall 

(0.23082191780821917, 0.903485254691689)

In [5]:
from sklearn import svm
clf = svm.SVC()
fold = 10

accuracy = cross_val_score(clf, features, labels, cv=fold)
precision = cross_val_score(clf, features, labels, cv=fold, scoring='precision')
recall = cross_val_score(clf, features, labels, cv=fold, scoring='recall')

print('Accuracy: ', accuracy.sum() / fold)
print('Precision: ', precision.sum() / fold)
print('Recall: ', recall.sum() / fold)

('Accuracy: ', 0.8856400985808689)
('Precision: ', 0.87857810569316952)
('Recall: ', 0.86510996424475484)
